In [ ]:
! git clone https://github.com/NVIDIA/apex
! cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [ ]:
! pip uninstall -y enum34
! pip install --quiet fuzzywuzzy
! pip install --quiet diskcache
! pip install --quiet python-Levenshtein
! pip install --quiet xxhash numba statsmodels xlrd toolz

In [ ]:
! pip install -e 'git://github.com/nandanrao/embed-software.git#egg=embed_software'

In [ ]:
! pip install --quiet --upgrade simpletransformers 

In [1]:
import transformers

In [2]:
import pandas as pd
import numpy as np

from validation.data import indeed_test_data, dot_train_data, get_soc_n
from embed_software.preprocess import *
from embed_software.utils import get_embeddings, embed_docs
from validation.dot_data import LemmaTokenizer, get_dictionary
from classification.embedding import PreEmbeddedVectorizer, Embedding, WordEmbeddingVectorizer
from validation.scoring import bubbleup_score, BubbleUpMixin

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
SAMPLE_SIZE = 50000
SOC_LEVEL = 6
BUBBLE_UP = 3
PROD = True

In [4]:
X_train, y_train = dot_train_data(SOC_LEVEL)
X_test, y_test, ids = indeed_test_data('../data/us/everything.csv', SAMPLE_SIZE, SOC_LEVEL)
if PROD == False:
    noprod_idx = get_soc_n(y_train.astype(str), 2) != 51
    X_train, y_train = X_train[noprod_idx], y_train[noprod_idx]

In [5]:
train_df = pd.DataFrame({ 'text': X_train.tolist(), 'label': y_train.tolist() })

train_df

,text,label
0,operates blindstitch sewing machine to join fa...,516031
1,performs as nonspeaking member of scene in sta...,272011
2,directs and coordinates through subordinate su...,111011
3,circumcises jewish male infants in accordance ...,212099
4,researches plans designs and administers build...,171011
...,...,...
32393,test vessels for leaks damage and defects and ...,537121
32394,unload cars containing liquids by connecting h...,537121
32395,copy and attach load specifications to loaded ...,537121
32396,start pumps and adjust valves or cables to reg...,537121


In [6]:
eval_df = pd.DataFrame({ 'text': X_test.tolist(), 'label': y_test.tolist() })

eval_df

,text,label
0,contract position available immediately must b...,151061
1,part time about pizza is an anti establishment...,518012
2,50 000 60 000 year we are looking for new exec...,351011
3,certified medical assistant unlv school of med...,319092
4,certified medical assistant unlv medicine the ...,319092
...,...,...
11163,we are currently looking for experienced veter...,392021
11164,60 000 150 000 year commission job title insid...,414012
11165,anita kurl salon fast paced growing salon in t...,391021
11166,working for and reporting directly to the firm...,434071


In [ ]:
from transformers.data.data

In [13]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained('../jobs-tokenizer')

In [14]:
import torch
from torch.utils.data.dataset import Dataset

class TrainDataset(Dataset):
    def __init__(self, tokenizer, X, y, block_size):

        batch_encoding = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=block_size, pad_to_max_length=True, return_atention_mask=True)

        self.ids = batch_encoding["input_ids"]
        self.mask = batch_encoding["attention_mask"]
        self.

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

In [17]:
train_data = TrainDataset(tokenizer, X_train.tolist(), 512)

In [20]:
tokenizer.batch_encode_plus(X_train.tolist()[:10]).keys()

dict_keys(['input_ids', 'attention_mask'])

In [21]:
torch.__version__

'1.5.0'

In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Train and Evaluation data needs to be in a Pandas Dataframe containing at least two columns. If the Dataframe has a header, it should contain a 'text' and a 'labels' column. If no header is present, the Dataframe should contain at least two columns, with the first column is the text with type str, and the second column in the label with type int.

# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base-uncased', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'fp16': False}) 
# You can set class weights by using the optional weight argument

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

# predictions, raw_outputs = model.predict(["Some arbitary sentence"])